In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
sys.path.append("..")  # Add parent directory to the system path
sys.path.append("/mnt/home/network-predictive-analysis/")
import config
from io import StringIO
from itertools import product


COLAB=False

if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    curr_path = config.drive_path
else:
    curr_path = os.getcwd()
    
data_path = os.path.join(curr_path, "../", config.data_path)
proc_data_path = os.path.join(curr_path, "../", config.processed_data_path)

In [ ]:
# retrieve the link_df, a df version of the dragonfly-link-stats file which holds link information 

link_file = os.path.join(data_path, "riodir-61218-1704940703", "dragonfly-link-stats")
lines = []

with open(link_file, 'r') as file:
    i = 0
    for line in file:
        if (line.startswith('#')):
            continue

        lines.append(line)
        i += 1

data_string = '\n'.join(lines)


link_df = pd.read_csv(StringIO(data_string), sep=' ', header=None)

link_df.columns = ['source_id', 'src_port_id', 'source_type', 'dest_id', 'dest_type', 'link_type', 'link_traffic', 'link_saturation', 'stalled_chunks']
link_df

In [ ]:
merged_df = pd.read_csv(os.path.join(proc_data_path, "merged_df_allPorts_scaled.csv"))
merged_df

In [ ]:
# create a tensor X which holds node features over time
# X: [num_timesteps, num_nodes, num_features]
# note: the 3rd axes (features) contain also the iteration-duration, the target for the prediction

X_array = []

num_nodes = int(merged_df['router-id'].max()) * 7 + merged_df['port-id'].max() + 1 #nodes of the graph dataset to be built, not computing nodes
iterations = merged_df['iteration'].max()

node_features_type = ['bw-consumed',
                 'downstream-credits-0', 'downstream-credits-1', 'downstream-credits-2', 'downstream-credits-3',
                 'vc-occupancy-0', 'vc-occupancy-1', 'vc-occupancy-2', 'vc-occupancy-3']

features_aggregation = ['-avg', '-q25', '-q75', '-min', '-max']

node_features = ['iteration-duration']
node_features.extend([a + b for a, b in product(node_features_type, features_aggregation)])

for graph_node_id in range(num_nodes):
  filtered_data = merged_df.loc[merged_df['router-id']*7 + merged_df['port-id'] == graph_node_id].reset_index().sort_values(by="iteration")
  node_features_over_time = np.transpose(filtered_data[node_features].values)
  X_array.append(node_features_over_time)
    

X = np.array(X_array) #252, 46, 2000
X = np.transpose(X, (2, 0, 1)) #2000, 252, 46
X.shape

In [ ]:
# features and target tensors have shape:
# features: [num_snapshots, num_timesteps_in, num_nodes, num_features]
# target: [num_snapshots, num_timesteps_in, num_nodes, num_features]
# each snapshot in the features tensor corresponds to a snapshot in the target tensor that we want to predict

num_timesteps_in = 50
num_timesteps_out = 50

indices = [(i, i + num_timesteps_in + num_timesteps_out) for i in range(iterations - num_timesteps_in - num_timesteps_out + 1)]

features, target = [], []
for lb, ub in indices:
  features.append((X[lb : lb + num_timesteps_in, :, :]))
  #target.append((X[:, 0, lb + num_timesteps_in : ub]))
  target.append((X[lb + num_timesteps_in : ub, :, :]))

features = np.array(features)
target = np.array(target)
features.shape, target.shape

In [ ]:
# train, val, test splits for features and targets

x_offsets = np.sort(np.concatenate((np.arange(-num_timesteps_in + 1, 1, 1),)))
y_offsets = np.sort(np.arange(1, num_timesteps_out + 1, 1))

train_ratio = 0.65
test_ratio = 0.2
# the rest (train_ratio - test_ratio) is used for the validation split

num_samples = features.shape[0]
num_test = round(num_samples * test_ratio)
num_train = round(num_samples * train_ratio)
num_val = num_samples - num_test - num_train

# train
x_train, y_train = features[:num_train], target[:num_train]

# val
x_val, y_val = (
    features[num_train: num_train + num_val],
    target[num_train: num_train + num_val],
)
# test
x_test, y_test = features[-num_test:], target[-num_test:]

for cat in ["train", "val", "test"]:
    _x, _y = locals()["x_" + cat], locals()["y_" + cat]

    print(cat, "x: ", _x.shape, "y:", _y.shape)

    store_path = os.path.join(proc_data_path, "SMART", f"in{num_timesteps_in}_out{num_timesteps_out}")
    
    if not os.path.exists(store_path):
        os.makedirs(store_path)

    np.savez_compressed(
        os.path.join(store_path, "%s.npz" % cat),
        x=_x,
        y=_y,
        x_offsets=x_offsets.reshape(list(x_offsets.shape) + [1]),
        y_offsets=y_offsets.reshape(list(y_offsets.shape) + [1]),
    )

In [ ]:
# construct sparse edge_index tensor
# [[n1, n2] [n1, n2] ...]

num_routers = int(merged_df['router-id'].max()) + 1
num_ports = int(merged_df['port-id'].max()) + 1
num_routers_per_group = 4
num_ports_per_router = 7

edge_index = []


# connect all the ports within the same router
# (edge_index) has 7*6 * 4 routers * 9 groups = 1512 edges after this step
for router_id in range(num_routers):
  for port_id_i in range(num_ports):
    for port_id_j in range(num_ports):
      if port_id_i != port_id_j:
        graph_node_id_i = router_id * num_ports_per_router + port_id_i
        graph_node_id_j = router_id * num_ports_per_router + port_id_j
        edge_index.append([graph_node_id_i, graph_node_id_j])
'''
  # id of the 1st router in the group the current router_id belongs to
  first_router_id = router_id % num_routers_per_group + router_id // num_routers_per_group * num_routers_per_group

  if first_router_id % num_routers_per_group == 0:
    # id of the 1st node (graph node, corresponding to the router's port) of the 1st router in the first group
    first_node_of_first_router_id = first_router_id * num_ports_per_router

    # id of the 1st node (graph node) in the current router
    first_node_id = router_id * num_ports_per_router

    inter_group_connections = [[2, 9], [3, 16], [4, 23], [10, 17], [11, 24], [18, 25]]
    for port_id_i, port_id_j in inter_group_connections:
      i = first_node_of_first_router_id + port_id_i
      j = first_node_of_first_router_id + port_id_j
      edge_index.append([i, j])
      edge_index.append([j, i])
'''

# this step extracts the local and global links from the link_df
# and results in 6*9*2 (local) + 8*9 (global) edges added to edge_index
link_df_filtered = link_df[(link_df['link_type'] == 'L') | (link_df['link_type'] == 'G')]
result_df = pd.merge(
    link_df_filtered,
    link_df_filtered,
    left_on=['dest_id', 'source_id', 'link_type'],
    right_on=['source_id', 'dest_id', 'link_type'],
    suffixes=('_s', '_d')
)
result_df['source_node_id'] = result_df['source_id_s'] * 7 + result_df['src_port_id_s']
result_df['destination_node_id'] = result_df['source_id_d'] * 7 + result_df['src_port_id_d']

edge_index.extend(result_df[['destination_node_id', 'source_node_id']].values)

len(edge_index)

In [ ]:
import torch
from torch_geometric.utils import to_dense_adj, dense_to_sparse
import pickle

# Convert sparse edge index to dense adjacency matrix
adj_dense = to_dense_adj(torch.tensor(edge_index).T)

shape = adj_dense.shape
adj_dense = adj_dense.reshape(shape[1], shape[2])

# Save to pickle file.
adj_matrix_file = os.path.join(proc_data_path, "adj.pkl")
with open(adj_matrix_file, 'wb') as f:
    pickle.dump([range(shape[0]), {i: i for i in range(shape[0])}, adj_dense], f, protocol=2)